In [2]:
import nltk
import os

nltk_data_path = "../data/libs/nltk_data"

os.makedirs(nltk_data_path, exist_ok=True)

nltk.data.path.append(nltk_data_path)

nltk.download('stopwords', download_dir=nltk_data_path)
# nltk.download('wordnet', download_dir=nltk_data_path)
# nltk.download('omw-1.4', download_dir=nltk_data_path)
# nltk.download('punkt_tab', download_dir=nltk_data_path)

print(f"NLTK Data Path: {nltk.data.path}")

NLTK Data Path: ['/home/miellilas/nltk_data', '/home/miellilas/Documents/pba/googleplay_garminconnect/venv/nltk_data', '/home/miellilas/Documents/pba/googleplay_garminconnect/venv/share/nltk_data', '/home/miellilas/Documents/pba/googleplay_garminconnect/venv/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', '../data/libs/nltk_data']


[nltk_data] Downloading package stopwords to ../data/libs/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
import pandas as pd
import re
import spacy
from nltk.corpus import stopwords
import datetime
import contractions

nlp = spacy.load('en_core_web_sm')
nltk_data_path = "../data/libs/nltk_data"
nltk.data.path.append(nltk_data_path)

In [6]:
garminconnect_df = pd.read_csv("../data/raw/20250313_1234_garminconnect_gplay_reviews.csv")
garminconnect_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,ff93503a-df4d-48ca-b1bf-4021a67900cb,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Hate the new layout. Trying to figure out how ...,2,2,5.10,2025-03-12 07:03:48,NaN,NaN,5.10
1,6e147f11-76a6-4971-bb4a-41d0d2a2f8d0,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Love it, helps me tremendously",5,0,5.10,2025-03-12 06:42:08,NaN,NaN,5.10
2,004dafcd-a440-434d-a052-a96b935a39df,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Very Good and carefree,5,0,5.10,2025-03-12 05:05:14,NaN,NaN,5.10
3,f52f11f3-8c6e-41b3-a17c-963cbfbb2ad8,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Latest update wont allow me to create a workou...,2,0,5.10,2025-03-12 03:59:57,NaN,NaN,5.10
4,8ebbecf7-da22-4bdd-b43a-4729494212d9,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"Much better than fitbit, no issues syncing my ...",5,0,5.10,2025-03-12 02:37:58,NaN,NaN,5.10


In [8]:
stop_words = set(stopwords.words('english'))
negation_words = {"no", "not", "nor", "never", "n't", "dont"}
stop_words = stop_words - negation_words


def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Lowercase
    text = text.lower()

    # Remove URLs, emails
    text = re.sub(r"http\S+|www\S+|\S+@\S+", "", text)

    # Remove mentions (e.g., @user), keep hashtags if needed
    text = re.sub(r"@\w+", "", text)

    # Keep apostrophes for contractions like don't, won't, etc.
    text = re.sub(r"[^a-z'\s]", "", text)

    # Tokenize and lemmatize
    doc = nlp(text)

    tokens = []
    for token in doc:
        lemma = token.lemma_.lower()

        # Keep alpha words or contractions like "don't"
        if lemma in negation_words:
            tokens.append(lemma)
        elif token.is_alpha and lemma not in stop_words and len(lemma) > 1:
            tokens.append(lemma)

    return " ".join(tokens)

In [15]:
stop_words = set(stopwords.words("english"))
negation_words = {"no", "not", "nor", "never", "n't", "dont"}
stop_words = stop_words - negation_words


# Optional dictionary for post-processing corrections (lemmatization errors, slang, etc.)
post_lemmatization_corrections = {
    "datum": "data",
    "cannot": "can_not",
    "dont": "do_not",
    "doesnt": "does_not",
    "wont": "will_not",
    "cant": "can_not",
    "isnt": "is_not",
    "wasnt": "was_not",
    "arent": "are_not"
}

def preprocess_text_v2(text):
    if not isinstance(text, str):
        return ""

    # Expand contractions: e.g., "don't" → "do not"
    text = contractions.fix(text)

    # Lowercase
    text = text.lower()

    # Remove URLs, emails
    text = re.sub(r"http\S+|www\S+|\S+@\S+", "", text)

    # Remove mentions
    text = re.sub(r"@\w+", "", text)

    # Replace exclamations with a tag (optional emphasis cue)
    text = re.sub(r"!", " EXCLAMATION ", text)

    # Remove non-alpha characters except apostrophes
    text = re.sub(r"[^a-z'\s]", "", text)

    # Tokenize & lemmatize
    doc = nlp(text)

    tokens = []
    skip_next = False

    for i, token in enumerate(doc):
        if skip_next:
            skip_next = False
            continue

        lemma = token.lemma_.lower()
        lemma = post_lemmatization_corrections.get(lemma, lemma)

        # Preserve negation + meaningful word (negation tagging)
        if lemma in negation_words and i + 1 < len(doc):
            next_token = doc[i + 1]
            if next_token.pos_ in {"ADJ", "VERB", "ADV", "NOUN"}:
                next_lemma = next_token.lemma_.lower()
                next_lemma = post_lemmatization_corrections.get(next_lemma, next_lemma)
                tokens.append(f"{lemma}_{next_lemma}")
                skip_next = True
                continue
            else:
                tokens.append(lemma)
        elif lemma not in stop_words and token.is_alpha and len(lemma) > 1:
            tokens.append(lemma)

    # Join into final cleaned string
    return " ".join(tokens)


### Sampling to test the preprocess_text function

In [16]:
sample_garminconnect_df = garminconnect_df.tail(10)

# sample_garminconnect_df['clean_content'] = sample_garminconnect_df['content'].apply(preprocess_text)
sample_garminconnect_df['clean_content'] = sample_garminconnect_df['content'].apply(preprocess_text_v2)

sample_garminconnect_df = sample_garminconnect_df[sample_garminconnect_df['clean_content'].str.strip().astype(bool)]

sample_garminconnect_df[["content", "clean_content"]]

/tmp/ipykernel_13699/2219113959.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_garminconnect_df['clean_content'] = sample_garminconnect_df['content'].apply(preprocess_text_v2)


,content,clean_content
125846,"I agree with quite a few people on here, how c...",agree quite people come not_upload course work...
125847,Very limited app... A real shame...,limited app real shame
125848,I had big hopes for this but it crashes all th...,big hope crash time samsung report calendar vi...
125849,Garmin please fix your app for the galaxy note...,garmin please fix app galaxy note keep turn de...
125850,"Same issue as Dan Percy. However, a factory r...",issue dan percy however factory reset not_work...
125851,The edge 810 works well until I pair it with e...,edge work well pair either htc samsung nexus p...
125852,"As soon as I launch Connect on my phone, my 81...",soon launch connect phone turn not_happen conn...
125853,Why no power data for cycling!!!!,no_power data cycling
125854,I can't seem to send a prepared course (on the...,not_seem send prepared course garmin connect w...
125855,dont work with 910 xt,not_work xt


### Full data cleaning

In [ ]:
garminconnect_df['clean_content'] = garminconnect_df['content'].apply(preprocess_text_v2)

garminconnect_df = garminconnect_df[garminconnect_df['clean_content'].str.strip().astype(bool)]

In [12]:
date_today = datetime.datetime.today().strftime("%Y%m%d_%H%M")
garminconnect_df.to_csv(f"../data/processed/{date_today}_garminconnect_df_cleaned.csv", index=False)

In [14]:
garminconnect_df[["content", "clean_content"]]

,content,clean_content
0,Hate the new layout. Trying to figure out how ...,hate new layout try figure load previous versi...
1,"Love it, helps me tremendously",love help tremendously
2,Very Good and carefree,good carefree
3,Latest update wont allow me to create a workou...,late update allow create workout category run ...
4,"Much better than fitbit, no issues syncing my ...",much well fitbit issue sync workout
...,...,...
125851,The edge 810 works well until I pair it with e...,edge work well pair either htc samsung nexus p...
125852,"As soon as I launch Connect on my phone, my 81...",soon launch connect phone turn happen connect ...
125853,Why no power data for cycling!!!!,power datum cycling
125854,I can't seem to send a prepared course (on the...,seem send prepared course garmin connect websi...
